In [1]:
# comment when exporting .py
def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

if is_colab():
  !git clone https://github.com/adam840502/absolute-bert.git
  import sys
  sys.path.append('absolute-bert')
  %cd absolute-bert/
  !git fetch
  !git checkout infra/wandb-logging
  !pip install --quiet numpy pandas tqdm nltk beir omegaconf datasets wandb faiss-cpu

fatal: destination path 'absolute-bert' already exists and is not an empty directory.
/content/absolute-bert
Already on 'infra/wandb-logging'
Your branch is up to date with 'origin/infra/wandb-logging'.


In [2]:
# comment when exporting .py
!mkdir -p configs

In [3]:
# {
#   'tokenizer_type': tokenizer_type,
#   'model': str(model),
#   'model_params': str(model_params),


#   # 'loss_type': str(using_loss),
#   'loss_params': loss_params,
# }

In [4]:
# comment when exporting .py
%%writefile configs/default.yaml
job_type: train
epochs: 1
masking_probability: 0.15
max_length: 256
random_seed: 42
validation_portion: 0.9
sampling_word_size: 100  # 用在 sampled softmax loss
lr: 1e-4
scheduler: cosine
warmup_ratio: 0.1
max_steps: -1
clip_loss: 50

model:
  depth: 12
  num_heads: 12
  dim: 768
  k_temperature: 0.5
  # hidden_dim: 32
  # embedding_initialize_method: 'rand'
  # attention_type: Absolute_global_attention
  # time_dim: 64

batch_sizes:
  train: 48
  val: 128
  IR: 32

log_intervals:
  train: 10
  val: 500
  params: 500
  IR: 2000

Overwriting configs/default.yaml


In [5]:
import argparse
import os
from omegaconf import OmegaConf

In [6]:
# # uncomment when exporting .py
# parser = argparse.ArgumentParser()
# parser.add_argument('--config', type=str, default='config.yaml')
# args = parser.parse_args()

# if os.path.exists(args.config):
#     config = OmegaConf.load(args.config)
# else:
#     config = OmegaConf.load("configs/default.yaml")  # 拆出來囉

In [7]:
# comment when exporting .py
config = OmegaConf.load("configs/default.yaml")
config

{'job_type': 'train', 'epochs': 1, 'masking_probability': 0.15, 'max_length': 256, 'random_seed': 42, 'validation_portion': 0.9, 'sampling_word_size': 100, 'lr': 0.0001, 'scheduler': 'cosine', 'warmup_ratio': 0.1, 'max_steps': -1, 'clip_loss': 50, 'model': {'depth': 12, 'num_heads': 12, 'dim': 768, 'k_temperature': 0.5}, 'batch_sizes': {'train': 48, 'val': 128, 'IR': 32}, 'log_intervals': {'train': 10, 'val': 500, 'params': 500, 'IR': 2000}}

In [8]:
# comment when exporting .py
dry_test = False

# 主程式

In [9]:
from tqdm.auto import tqdm

from datasets import load_from_disk

import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, DataCollatorForLanguageModeling

import wandb

import absolute_bert.absolute_bert_outputAct as abs_bert
import absolute_bert.mlm_losses as losses

import torch.nn as nn
from utils.evaluate import BeirBenchmark, BI_ENCODER_METHODS
from utils.train import format_losses, MultiLossAverager
from utils.logging import extract_param_stats

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [11]:
from transformers import get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup

schedulers = {
  'cosine': get_cosine_schedule_with_warmup,
  'linear': get_linear_schedule_with_warmup
}

In [12]:
run = wandb.init(entity = "ghuang-nlp",
  project = "prototyping",
  job_type = config.job_type,
  group = config.scheduler,
  tags = [config.job_type, config.scheduler],
  config = OmegaConf.to_container(config, resolve=True),
  mode = 'disabled' if dry_test else 'online',
  save_code = True)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: adam840502 (ghuang-nlp) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [13]:
artifact = run.use_artifact('ghuang-nlp/uncategorized/wikipedia.en-0.01:v0', type='dataset')
artifact_dir = artifact.download()
datadict = load_from_disk(artifact_dir)
print(datadict)

wandb: Downloading large artifact wikipedia.en-0.01:v0, 741.71MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:1.0


DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 1523836
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 15141
    })
})


In [14]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=config.masking_probability)
train_loader = DataLoader(datadict['train'], batch_size=config.batch_sizes.train, collate_fn=collator)
val_loader = DataLoader(datadict['test'], batch_size=config.batch_sizes.val, collate_fn=collator)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [15]:
model = abs_bert.Absolute_bert_for_masked_LM(tokenizer.vocab_size, **config.model).to(device)
loss_fn = losses.Cross_entropy(model)
print(model)

Absolute_bert_for_masked_LM(
  (base_model): Absolute_bert(
    (embedding): Embedding(30522, 768)
    (layers): ModuleList(
      (0-7): 8 x Absolute_attention(
        (Q): Linear(in_features=768, out_features=768, bias=False)
        (K): Linear(in_features=768, out_features=768, bias=True)
        (V): Linear(in_features=768, out_features=768, bias=True)
        (O): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.5, inplace=False)
        (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (linear_in): Linear(in_features=768, out_features=1536, bias=True)
        (act_fn): GELU(approximate='tanh')
        (linear_out): Linear(in_features=1536, out_features=768, bias=True)
        (layer_norm_lin): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      )
    )
  )
)


In [16]:
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
  {
    "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
    "weight_decay": 0.01,
  },
  {
    "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
    "weight_decay": 0.0,
  },
]
optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=config.lr)

In [17]:
scheduler = schedulers[config.scheduler](
  optimizer,
  num_warmup_steps = len(train_loader) * config.epochs * config.warmup_ratio,
  num_training_steps = len(train_loader) * config.epochs,
)

In [18]:
def pure_text_token_vectors(output, inputs):
  """special_token and padding_token will map to zero vector."""
  mask = (inputs['attention_mask'] * (1 - inputs['special_tokens_mask'])).bool()
  return output.masked_fill(~mask[:, :, None], 0.0)

def tokenize_fn(texts):
  inputs = tokenizer(texts, return_tensors='pt', padding=True, return_special_tokens_mask=True)
  special_tokens_mask = inputs.pop('special_tokens_mask')
  return inputs, {'special_tokens_mask': special_tokens_mask}

model_output_method = BI_ENCODER_METHODS['bert_like_siamese_simple_pool'](
  model,
  tokenize_fn = tokenize_fn,
  output_key = 0,
  common_post_fn = pure_text_token_vectors,
  aggregate_method = 'mean'
)

class StaticEmbeddingsWithTransform(nn.Module):
  def __init__(self, model):
    super().__init__()
    self.model = model

  def forward(self, **inputs):
    return self.model.base_model.embedding(inputs['input_ids'])

static_embeddings_method = BI_ENCODER_METHODS['bert_like_siamese_simple_pool'](
  StaticEmbeddingsWithTransform(model).to(device),
  tokenize_fn = tokenize_fn,
  common_post_fn = pure_text_token_vectors,
  aggregate_method = 'mean'
)

In [19]:
benchmark = BeirBenchmark(corpus_name='scifact', batch_size=config.batch_sizes.IR)

# model_output_metrics = benchmark.run(model_output_method)
# static_embeddings_metrics = benchmark.run(static_embeddings_method)

  0%|          | 0/5183 [00:00<?, ?it/s]

In [20]:
def get_beir_log_dict(metric_dict, method_name):
  return \
    {f"{benchmark.corpus_name}-{method_name}/{k}": v
      for k, v in metric_dict.items()} | \
    {f"highlight-{benchmark.corpus_name}-{method_name}/{k}": v['10']
      for k, v in metric_dict.items()}

In [21]:
averager = MultiLossAverager()
global_step = 0

model.eval()
wandb.log(
  get_beir_log_dict(benchmark.run(model_output_method), 'model_output'),
  step=global_step
)
wandb.log(
  get_beir_log_dict(benchmark.run(static_embeddings_method), 'static_embeddings'),
  step=global_step
)
wandb.log(extract_param_stats(
        [
          model.base_model.layers[0].layer_norm,
          model.base_model.layers[-1].layer_norm,
        ],
        model
      ))


for epoch_num in range(config.epochs):

  bar = tqdm(train_loader)
  for batch_num, batch in enumerate(bar):

    model.train()
    optimizer.zero_grad()

    batch = {key: batch[key].to(device) for key in batch}
    predicts, targets = model(**batch)

    loss = loss_fn(predicts, targets)
    final_loss, loss_dict = format_losses(loss, clip_value=config.clip_loss)

    if global_step % config.log_intervals.train == 0:
      wandb.log({f"train/{k}": v for k, v in loss_dict.items()}, step=global_step)

    bar.set_postfix(loss_dict)

    # with torch.autograd.detect_anomaly(True):
      # loss.backward()
      # torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=10, norm_type=2)

    final_loss.backward()
    optimizer.step()
    global_step += 1
    scheduler.step()

    if global_step % config.log_intervals.val == 0:
      model.eval()

      with torch.no_grad():
        val_bar = tqdm(val_loader, leave=False)
        for batch in val_bar:
          batch = {key: batch[key].to(device) for key in batch}
          predicts, targets = model(**batch)
          loss = loss_fn(predicts, targets)
          _, loss_dict = format_losses(loss, clip_value=config.clip_loss)

          batch_size = batch["labels"].size(0)
          averager.update(loss_dict, batch_size)

          val_bar.set_postfix(loss_dict)

      avg_losses = averager.compute()
      wandb.log({f"val/{k}": v for k, v in avg_losses.items()}, step=global_step)
      averager.reset()

    if global_step % config.log_intervals.params == 0:
      model.eval()
      wandb.log(extract_param_stats(
        [
          model.base_model.layers[0].layer_norm,
          model.base_model.layers[-1].layer_norm,
        ],
        model
      ))

    if global_step % config.log_intervals.IR == 0:
      model.eval()
      wandb.log(
        get_beir_log_dict(benchmark.run(model_output_method), 'model_output'),
        step=global_step
      )
      wandb.log(
        get_beir_log_dict(benchmark.run(static_embeddings_method), 'static_embeddings'),
        step=global_step
      )


    # del batch, loss

    if (config.max_steps > 0) and (global_step > config.max_steps):
      break

  model.eval()
  wandb.log(
    get_beir_log_dict(benchmark.run(model_output_method), 'model_output') | {'epoch': epoch_num},
    step=global_step
  )
  wandb.log(
    get_beir_log_dict(benchmark.run(static_embeddings_method), 'static_embeddings') | {'epoch': epoch_num},
    step=global_step
  )

  0%|          | 0/31747 [00:00<?, ?it/s]

  0%|          | 0/119 [00:00<?, ?it/s]

wandb: WARNING Tried to log to step 500 that is less than the current step 501. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.


KeyboardInterrupt: 

In [ ]:
# model_artifact = wandb.Artifact(name="model",
#   type="model",
#   # description="trained with 2-1-training_with_msmarco",
#   metadata=training_args_config)
# model_artifact.add_dir(saving_dir)
# run.log_artifact(model_artifact)

# run.log_code(**get_code_files_aggregating_functions(cfg.env.project_root))

In [ ]:
# if not cfg.testing:
#     if not os.path.exists(saving_dir):
#         os.mkdir(saving_dir)

#     trainer.save_model(saving_dir)
#     trainer.save_state()
#     torch.save(model, saving_dir/'pytorch.pt')

### for markdown recording

In [ ]:
# for polaritical_docs in zip(test_cossim_before, test_cossim_after):  # pos & neg
#     print(f"||{'|'.join(str(idx) for idx in range(10))}|")
#     for timepoint, values in zip(['before', 'after'], polaritical_docs):  # before & after
#         print(f"|{timepoint}|{'|'.join(f'{val:.4f}' for val in values.tolist())}|")
#     print()

In [ ]:
run.finish()